In [5]:
%%pyspark
# Generating interactions

from pyspark.sql.functions import *
from pyspark.sql import types
from random import randrange
import datetime

timestamp_udf = udf(lambda x: datetime.datetime.now() + datetime.timedelta(days = -int(x), hours = -int(x), minutes = -int(x)), types.TimestampType())
df1 = spark.sql("select id as person_id from dataverse_contosodefau_unq35c64bd569f74356a67a71fce467b.account limit 1000") #random 1000 users

#100 -> 100(2^i)
for _ in range(5):
    df1 = df1.union(df1).sample(False, 0.6).coalesce(10)
df1 = df1.orderBy(rand()) #shuffled

#generate a random timestamp
df1 = df1.withColumn("offset", round(rand()*100,0))
df1 = df1.withColumn("interaction_date", timestamp_udf(df1['offset']))
#df1.createOrReplaceGlobalTempView("interactions")
df1.sample(False, 0.8).select("person_id", "interaction_date").coalesce(10).write.parquet("interactions", mode = 'overwrite')

StatementMeta(SparkPool1, 0, 8, Finished, Available)